Since `predict_embedding` doesn't work the way I'd wish (I probably need LabelEncoder or something), I've tried to re-do the entire section based on this notebook: https://www.kaggle.com/code/aerdem4/h-m-pure-pytorch-baseline

In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [11]:
# Load data
df = pd.read_csv("dataset/transactions_train.csv", dtype={"article_id": str})

In [14]:
art_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])
# TODO I don't why we concat placeholder - can we remove it as well?
art_ids = np.unique(df["article_id"].values)
art_ids

array(['0108775015', '0108775044', '0108775051', ..., '0953450001',
       '0953763001', '0956217002'], dtype=object)

In [15]:
art_LE = LabelEncoder()
art_LE.fit(art_ids)
test = art_LE.transform(df["article_id"])
test

array([ 40140,  10505,   6377, ..., 103303,  88328, 101162])

In [33]:
testdf = pd.DataFrame({'A': [1,2,3], 'B': [4,5,6]})
testdf

,A,B
0,1,4
1,2,5
2,3,6


In [105]:
df_np = df.head(10).to_numpy()
sample = df_np[1]
np.where((df_np[:, 1] == sample[1]) & (df_np[:, 2] == sample[2]))[0].shape[0]

1

In [80]:
ret_df = pd.DataFrame(columns=testdf.columns)
for i in range(10):
    samp = np.array([testdf[:,j].sample().values for col in testdf.columns]).flatten()
    ret_df.loc[i] = samp
# ret_df

ret_df.to_numpy()

array([[3, 4],
       [2, 5],
       [2, 6],
       [3, 6],
       [3, 6],
       [1, 5],
       [2, 5],
       [3, 5],
       [2, 4],
       [2, 4]], dtype=int64)

In [72]:
%reload_ext jupyter_black

In [119]:
## Create dataset


def create_dataset(df, total_cases: int, portion_positive: float = 0.1):
    n_cases_train = round(total_cases * portion_positive)
    n_cases_val = total_cases - n_cases_train

    train_df = df.sample(n_cases_train).reset_index(drop=True)

    # Generate false samples
    df_np = df.to_numpy()
    val_np = np.empty((n_cases_val, df_np.shape[1]), dtype=object)
    for i in range(n_cases_val):
        legit_negative = False
        # Assure sample is not actually real by checking ID tuples to true df
        while not legit_negative:
            sample = np.array(
                [np.random.choice(df_np[:, col]) for col in range(df_np.shape[1])]
            )
            legit_negative = (
                np.where((df_np[:, 1] == sample[1]) & (df_np[:, 2] == sample[2]))[
                    0
                ].shape[0]
                == 0
            )
        val_np[i, :] = sample

    return train_df, pd.DataFrame(data=val_np, columns=df.columns)


train_df, val_df = create_dataset(df, total_cases=100)

In [124]:
import torch

# torch.zeros(2).float()*
# len(art_ids)*
row = df.iloc[4]
target = torch.zeros(len(art_ids)).float()
for t in row.target:
    print(t)
    print(row.target)

AttributeError: 'Series' object has no attribute 'target'

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    def __init__(self, df, seq_len, is_test=False):
        self.df = df#.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        if self.is_test:
            target = torch.zeros(2).float()
        else:
            target = torch.zeros(len(art_ids)).float() # TODO pass art_ids as parameter?
            for t in row.target:
                target[t] = 1.0
            
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()
        
        
        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            else:
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2
                
        return article_hist, week_hist, target
    
HMDataset(val_df, 64)[1]